In [21]:
import numpy as np               
import matplotlib.pyplot as plt                       
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas.tools import sjoin
import rioxarray as rx
import cartopy.crs as ccrs
import pyarrow

In [ ]:
#Load Storm Surge Advisory 1 data
ssa1_shp  = gpd.read_file(f"SSA1/PH_SSA1.shp")

In [ ]:
# Load asset polygons
asset_polygons = gpd.read_shp("sample_locs.shp")

# Ensure the CRS is EPSG:4326
asset_polygons = asset_polygons.to_crs(epsg=4326)

In [24]:
# Spatial join: asset polygons that intersect with SSA1
df_sjoin = gpd.sjoin(asset_polygons, ssa1_shp, how='inner', predicate='intersects')

# Keep necessary columns
df_sjoin = df_sjoin[["Facility Name", "LAT", "LON", "geometry"]]  # update to your actual column names
df_sjoin = df_sjoin.drop_duplicates(subset=['LAT', 'LON']).reset_index(drop=True)

# Save simplified DataFrame for plotting/analysis
df_sjoin_point_LATLON = df_sjoin[["Facility Name", "LON", "LAT"]]

In [26]:
ds_slr = xr.open_dataset(f"slr_projections/total_{year}_ssp{SSP}_Medium.nc")
print(ds_slr.dims)
print(ds_slr.coords)

FrozenMappingWarningOnValuesAccess({'CI': 3, 'lat': 181, 'lon': 360})
Coordinates:
  * lon      (lon) float32 1kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat      (lat) float32 724B 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
  * CI       (CI) float32 12B 0.05 0.5 0.95


In [28]:
# get SLR projetions for facility locations, different future decades, and projection quantiles
quant_list = [0.05, 0.5 ,0.95]
year_list = np.arange(2030,2051,10)     
SSP = '245'

for i, year in enumerate(year_list):
    for j, quant in enumerate(quant_list):
        #Sea level rise projections and SRTM DEM
        ds_slr = xr.open_dataset(f"slr_projections/total_{year}_ssp{SSP}_Medium.nc")
        ds_slr = ds_slr.sel(lon=slice(115.0,128.0),lat=np.arange(3,22))
        #print(np.max(ds_slr.sealevel_mm.values))

        #Select quantile and remove missing values
        ds_slr = ds_slr.sel(CI=quant)
        ds_slr = ds_slr.where(ds_slr['sealevel_mm'] > -32768)  
        
        # create dataframe for saving CSV output file
        quant_col = f"{year} Sea Level Rise CI {quant}"
        cols = ["Facility", "Lat", "Lon", quant_col]
        df_quant = pd.DataFrame(columns=cols)

        # Loop through facility locations
        for row in df_sjoin_point_LATLON.itertuples():
            print(row)

            slr_proj = ds_slr.sel(lon=row.LON, lat=row.LAT, method="nearest").sealevel_mm.values / 1000 # convert to meters

        # for very first entry, set dataframe
        if i==0 and j==0:
            df_master = df_quant
        # else concat column-wise
        else:
            df_master = pd.concat([df_master, df_quant[quant_col]], axis=1)
        
# Save to csv    
print(df_master)
df_master.to_csv(f"slr_SSP{SSP}.csv", index=False)

Pandas(Index=0, _1='Corona Del Mar Phase 1', LON=123.81705, LAT=10.23965)
Pandas(Index=1, _1='Columna', LON=120.97601, LAT=14.59826)
Pandas(Index=2, _1='Marina Town Mall (Dumaguete)', LON=123.31199, LAT=9.32254)
Pandas(Index=3, _1='Crimson Boracay', LON=121.90884, LAT=11.98249)
Pandas(Index=4, _1='Crimson Mactan', LON=124.0141, LAT=10.29674)
Pandas(Index=5, _1='Villa Mercedita', LON=125.52824, LAT=7.03366)
Pandas(Index=6, _1='Ocean Cove', LON=125.5323, LAT=7.03592)
Pandas(Index=7, _1='Centro Spatial Davao', LON=125.60833, LAT=7.06194)
Pandas(Index=8, _1='Kembali Coast', LON=125.73083, LAT=6.9502)
Pandas(Index=9, _1='Futura Shores Dumaguete', LON=123.30569, LAT=9.28803)
Pandas(Index=10, _1='Dauin Negros Hospitality', LON=123.24381, LAT=9.16161)
Pandas(Index=11, _1='Quest Puerto Hospitality', LON=118.75477, LAT=9.72947)
Pandas(Index=12, _1='Bohol Hospitality', LON=123.86052, LAT=9.59533)
Pandas(Index=13, _1='FDC Misamis Power Plant', LON=124.74937, LAT=8.55864)
Pandas(Index=0, _1='Corona

In [34]:
quant_list = [0.05, 0.5, 0.95]
year_list = np.arange(2030, 2051, 10)
ssp_list = ['245', '585']  

cols_base = ["Facility Name", "LAT", "LON"]

for SSP in ssp_list:
    df_master = df_sjoin_point_LATLON[cols_base].copy()

    for year in year_list:
        for quant in quant_list:
            # Load NetCDF file for current SSP and year
            file_path = f"slr_projections/total_{year}_ssp{SSP}_Medium.nc"
            try:
                ds_slr = xr.open_dataset(file_path)
            except FileNotFoundError:
                print(f"Missing: {file_path}")
                continue

            ds_slr = ds_slr.sel(lon=slice(115.0, 128.0), lat=np.arange(3, 22))

            # Select quantile and filter no-data values
            ds_slr = ds_slr.sel(CI=quant)
            ds_slr = ds_slr.where(ds_slr['sealevel_mm'] > -32768)

            # Create new column name
            quant_col = f"{year}_SLR_CI_{quant}"
            slr_values = []

            for row in df_sjoin_point_LATLON.itertuples():
                slr_proj = round(ds_slr.sel(lon=row.LON, lat=row.LAT, method="nearest").sealevel_mm.values / 1000, 2)  # mm to m
                slr_values.append(slr_proj)

            # Add column to df_master
            df_master[quant_col] = slr_values

    # Save results for each SSP separately
    output_file = f"slr_SSP{SSP}.csv"
    print(f"Saving: {output_file}")
    df_master.to_csv(output_file, index=False)

Saving: slr_SSP245.csv
Saving: slr_SSP585.csv
